<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark Setting

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 45.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=17aed0a6c4c9a7445a59033ffbabdab7cc7bcd15220b3ff4ff2f594f780e568e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
spark

# Libraries

In [4]:
import pandas as pd
import numpy as np

In [63]:
from pyspark.sql.types import FloatType 
from pyspark.sql.functions import min, udf, lit, col
import pyspark.sql.functions as f
from pyspark.sql import Window

# Dataset

In [6]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 41.6MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
   creating: Bigdata_hw2_datasets/
   creating: Bigdata_hw2_datasets/q1/
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
   creating: Bigdata_hw2_datasets/q2/
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
   creating: Bigdata_hw2_datasets/q3/
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [52]:
dataset_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/data.txt", header=None, names=["data"])
dataset_pandas["data"] = dataset_pandas["data"].apply(lambda x:x.split(" "))
dataset_pandas["data"] = dataset_pandas["data"].apply(lambda item: [float(x) for x in item])

c1_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/c1.txt", header=None, names=["center"])
c1_pandas["center"] = c1_pandas["center"].apply(lambda x:x.split(" "))
c1_pandas["center"] = c1_pandas["center"].apply(lambda item: [float(x) for x in item])

c2_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/c2.txt", header=None, names=["center"])
c2_pandas["center"] = c2_pandas["center"].apply(lambda x:x.split(" "))
c2_pandas["center"] = c2_pandas["center"].apply(lambda item: [float(x) for x in item])

In [53]:
dataset_pandas["data-id"] = range(len(dataset_pandas))
c1_pandas["center-id"] = range(len(c1_pandas))

In [54]:
dataset_spark = spark.createDataFrame(dataset_pandas)
center_spark = spark.createDataFrame(c1_pandas)

In [32]:
NUMBER_FEATURES = 58

# KMeans

In [33]:
def euclidean_distance(a, b):
  distance = 0
  for i in range(NUMBER_FEATURES):
    distance += (a[i] - b[i]) ** 2
  return distance ** 0.5

In [56]:
data_center_spark = dataset_spark.join(center_spark).withColumn('distance', euclidean_distance(dataset_spark.data, center_spark.center))

In [69]:
data_center_spark = data_center_spark.withColumn("min-distance", f.min("distance").over(Window.partitionBy("data-id")))\
                                    .where(col("min-distance") == f.col("distance"))\
                                    .drop(*["distance", "min-distance"])